In [167]:
import torch
import pandas as pd
import numpy as np
#import re
#import nltk
#nltk.download()

from transformers import *
import sys
sys.path.append('../py')

from bioinferdataset import BioInferDataset
from config import XML_PATH, PREPPED_DATA_PATH

In [ ]:
ber = "../data/text_sentences.txt"
data = pd.read_csv(ber, delimiter='\n', header=None)

dataset = BioInferDataset(XML_PATH)

dataset.load_samples_from_pickle(PREPPED_DATA_PATH)

dataset.load_samples_from_pickle(PREPPED_DATA_PATH)

In [184]:
sample = dataset.sample_list[0]
bert_tokens = sample["bert_tokens"]
text = sample["text"]
mask = sample["mask"]

In [199]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
def Bert_Tokens(sentence):
        tokenized = tokenizer.encode_plus(
                            text=sentence,  # the sentence to be encoded
                            add_special_tokens=True,  # Add [CLS] and [SEP]
#                             truncation=True,
#                             max_length = 40,  # maximum length of a sentence
#                             pad_to_max_length=True,  # Add [PAD]s
                            is_split_into_words=True,
#                             return_attention_mask = True,  # Generate the attention mask
                            return_offsets_mapping=True,
#                             return_length=True,
                        )
        return tokenized

In [196]:
sentence = data[0][0]
word_split_sentence = sentence.split(' ')

In [197]:
results = Bert_Tokens(word_split_sentence)

In [200]:
results

{'input_ids': [102, 6010, 579, 1793, 12280, 9233, 6130, 579, 1793, 12280, 3354, 214, 9778, 2256, 131, 106, 6130, 579, 1793, 12280, 1375, 105, 579, 377, 1491, 1375, 1732, 1127, 205, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 5), (5, 6), (6, 9), (9, 13), (0, 8), (0, 4), (4, 5), (5, 8), (8, 12), (0, 9), (0, 2), (0, 10), (0, 9), (0, 2), (0, 1), (0, 4), (4, 5), (5, 8), (8, 12), (12, 13), (13, 14), (14, 15), (15, 19), (0, 6), (6, 7), (7, 10), (0, 7), (0, 1), (0, 0)]}

In [186]:
marked_text = "[CLS] " + text+ " [SEP]"
# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)
tt = tokenized_text
# Print out the tokens.
print(tokenized_text)
print(len(tokenized_text))

['[CLS]', 'alpha', '-', 'cat', '##enin', 'inhibits', 'beta', '-', 'cat', '##enin', 'signaling', 'by', 'preventing', 'formation', 'of', 'a', 'beta', '-', 'cat', '##enin', '*', 't', '-', 'cell', 'factor', '*', 'dna', 'complex', '.', '[SEP]']
30


In [206]:
def parse_bert(seq_original, seq_bert):
    """
    """

    def remove_leading_pounds(token):
        """
        """
        token_new = ''
        for c in token:
            if c != '#':
                token_new += c
        return token_new

    # Remove header and footer tags.
#     seq_bert = seq_bert[1:-1])
    # Iterate over the original sequence and detect splitted tokens.
    mapped_indices_list = []
    j = 0
    for i in range(len(seq_original)):
        if seq_original[i] == seq_bert[j]:  # Not splitted.
            j += 1
            continue
        else:  # Detect splitted tokens.
            start = 0
            token_splitted = seq_original[i]
            token_mapping = remove_leading_pounds(seq_bert[j])
            mapped_indices = []
            while token_mapping == \
                    token_splitted[start : start + len(token_mapping)]:
                mapped_indices.append(j)
                start += len(token_mapping)
                j += 1
                token_mapping = remove_leading_pounds(seq_bert[j])
            mapped_indices_list.append((i, mapped_indices))
    return mapped_indices_list

In [207]:
tokenizer = AutoTokenizer.from_pretrained(
    'allenai/scibert_scivocab_uncased')

text_test = data[0][0]
seq_original = [w.lower() for w in data[0][0].split(' ')]
seq_bert = tokenizer.tokenize(text_test )

splits = parse_bert(seq_original, seq_bert)

In [15]:
def Bert_New_Embedding(bert,split,shape):
    '''
    '''
    print((split))
    j=0
    k=0
    bert_new=torch.zeros([1,shape,768])
    if len(split) != 0:
        for i in range(len(split)):
            while k< split[i][0]:
                bert_new[:,k,:]=bert[:,j,:]
                j += 1
                k += 1
            for p in range(len(split[i][1])):
                j += 1

            bert_new[:,k,:] = torch.sum(bert[:,split[i][1],:],dim = 1)/len(split[i][1])
            k +=1
        while k < shape:
            bert_new[:,k,:]=bert[:,j,:]
            j += 1
            k += 1
    else:
        bert_new = bert
        
    return bert_new

In [16]:
config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=config)
with torch.no_grad():
    outputs1 = model(input_ids,attention_mask )

In [178]:
attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [179]:
bert_tokens

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]])

In [190]:
mask

torch.Size([1, 147])

In [192]:
bert_enc = BERTEncoder(output_bert_hidden_states=False)
print(bert_enc.forward(bert_tokens.unsqueeze(0), mask.unsqueeze(0), text))

loading pretrained BERT...
([tensor([[[ 2.9975e-01,  1.1368e-01, -3.7701e-04,  ..., -2.5679e-01,
          -9.6255e-01, -6.1292e-01]],

        [[ 4.2694e-02,  5.1686e-01, -1.6080e-01,  ..., -1.1505e+00,
          -1.7851e+00, -6.7494e-01]],

        [[ 2.0229e-02,  2.8722e-01, -2.1754e-01,  ..., -3.4749e-01,
          -3.3012e-01,  6.4973e-01]],

        ...,

        [[ 6.0300e-01, -1.5243e-01, -9.8899e-01,  ..., -1.0068e+00,
           5.4732e-01, -4.0833e-01]],

        [[-1.2065e-01, -1.1065e+00, -2.5464e-01,  ...,  3.5409e-02,
           8.4949e-01, -7.2837e-01]],

        [[ 1.0213e+00,  4.8996e-01, -4.8651e-02,  ..., -1.8160e+00,
          -1.1686e+00, -6.7990e-01]]], grad_fn=<PermuteBackward>)], [28])


In [180]:
text

'alpha-catenin inhibits beta-catenin signaling by preventing formation of a beta-catenin*T-cell factor*DNA complex .'

In [211]:
bert_out=outputs1[0]
bert_out = bert_out[:,0:a,:]
bert_out = bert_out[:,1:-1,:]
bert_out.shape

torch.Size([1, 28, 768])

In [212]:
bert_out = Bert_New_Embedding(bert_out,splits,13)
bert_out = bert_out.permute(1, 0, 2)

[(0, [0, 1, 2, 3]), (2, [5, 6, 7, 8]), (9, [15, 16, 17, 18, 19, 20, 21, 22]), (10, [23, 24, 25])]


In [213]:
torch.save(bert_out,'../data/unit_tests/bert_out.tensor')

In [201]:
bert_splits

[4, 1, 4, 1, 1, 1, 1, 1, 1, 8, 3, 1, 1]